In [33]:
import os
import pandas as pd

import torch
from torch.utils.data import DataLoader
from PIL import Image

from torchvision.io import read_image

from torch.utils.data import Dataset
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

In [34]:
class CustomImageDataset(Dataset):
    def __init__(self, data, targets, transform=None, target_transform=None):# deve estrarre tutte 2 le versioni augmentate
        self.imgs = data
        self.targets = targets
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):# called each time you look at the lenght of the dataset
        return len(self.imgs)

    def __getitem__(self, idx):#crea a runtume le versioni augmentate
        img = self.imgs[idx]
        if isinstance(img, str):
          img = read_image(img)#working with paths
        else:
           image = Image.fromarray(img.astype('uint8'), 'RGB')
        label = self.targets[idx]
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            label = self.target_transform(label)
        return img, label

In [38]:
data = torchvision.datasets.CIFAR10(root='./data', train=True, download=True)

# simclr DA pipeline
s=1
size = 32 # should be the same as CIFRAR10, you can use it to do scale of CIFRAR10
color_jitter = transforms.ColorJitter(0.8 * s, 0.8 * s, 0.8 * s, 0.2 * s)
transform = transforms.Compose([transforms.RandomResizedCrop(size=size),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomApply([color_jitter], p=0.8),
                                  transforms.RandomGrayscale(p=0.2),
                                  transforms.GaussianBlur(kernel_size=int(0.1 * size)),
                                  transforms.ToTensor()])

# create training set from CustomDataset
trainset = CustomImageDataset(data=data.data, targets=data.targets, transform=transform, target_transform=transforms.ToTensor())


Files already downloaded and verified


In [39]:
dataloader = DataLoader(trainset, batch_size=64, shuffle=True)

In [40]:
# Display image and label.
from matplotlib import pyplot as plt

train_features, train_labels = next(iter(dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")


# use this
notransform = transforms.Compose([transforms.ToTensor()])
non_augmented = torchvision.datasets.CIFAR10(root='./data', train=True, transform=notransform)


from torchvision.utils import make_grid
imgs = torch.stack((*[non_augmented[i][0] for i in range(10)],
                    *[trainset[i][0] for i in range(10)],
                    *[trainset[i][1] for i in range(10)]))
grid = make_grid(imgs, nrow=10)

transforms.ToPILImage()(grid)

TypeError: Unexpected type <class 'numpy.ndarray'>

## Exercise 1
Create the custom training set using the cifar10 images and targets

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, data, targets, transform=None, target_transform=None):
        self.imgs = data
        self.targets = targets
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        img = self.imgs[idx]
        if isinstance(img, str):
          img = read_image(img)
        label = self.targets[idx]
        if self.transform:
            img = self.transform(img)
        if self.target_transform:
            label = self.target_transform(label)
        return img, label# sol esercizio 2 -> chiama 2 volte la transformazione e ritorna le 2 imgs

## Exercise 2
Modify the custom dataset to return two different views of an input image and create a train loop as the following example

In [ ]:
dataloader = DataLoader(trainset, batch_size=64, shuffle=True)

for idx, data in enumerate(dataloader):
    images, targets = data
    print(images.shape)
    print(targets.shape)
    if idx == 3:
        break

## Exercise 3

Plot a positive pair and a negative pair of a mini-batch of sample